In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

In [2]:
class NeuronalNet(nn.Module):
        def __init__(self):
            super(NeuronalNet, self).__init__()
            self.lin1 = nn.Linear(18,30)
            self.lin2 = nn.Linear(30,15)
            self.lin3 = nn.Linear(15,3)
        def forward(self,x):
            x = self.lin1(x)
            x = F.relu(x)
            x = F.relu(self.lin2(x))
            x = F.relu(self.lin3(x))
            return x

In [3]:
net = NeuronalNet()

In [ ]:
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/topTeamsEngland.csv
#'C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\topTeamsEngland.csv'

In [5]:
topTeams_england = pd.read_csv('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\topTeamsEngland.csv', sep=';')

In [6]:
df = pd.DataFrame(topTeams_england, columns = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR'])

In [7]:
df_names = df.drop_duplicates(['HomeTeam','AwayTeam'],keep = 'last')

In [8]:
df_names = df_names[['HomeTeam','AwayTeam']]

In [9]:
gameList = []
for row in df_names.itertuples():
    data = [[row.HomeTeam,row.AwayTeam]]
    gameList.append(pd.DataFrame(data))

In [10]:
df_names

,HomeTeam,AwayTeam
378,Arsenal,Man City
379,Chelsea,Arsenal
380,Man United,Tottenham
381,Tottenham,Liverpool
382,Arsenal,Everton
383,Chelsea,Liverpool
384,Liverpool,Man City
385,Chelsea,Man United
386,Man United,Everton
387,Tottenham,Man City


In [11]:
def inputValues(gameList, df_stats,inputList):
    for match in gameList:
        homeTeam = match.iat[0,0]
        awayTeam = match.iat[0,1]
        input_values= df_stats[df_stats['HomeTeam']==homeTeam]
        input_values= input_values[input_values['AwayTeam']==awayTeam]
        input_values= input_values.reset_index(drop=True)
        input_values = input_values[:-1]
        inputList.append(input_values.loc[:,['FTHG','FTAG']])

In [12]:
unnormaleListe = []
inputValues(gameList, df, unnormaleListe)
print(unnormaleListe[0])

   FTHG  FTAG
0     0     0
1     0     0
2     1     0
3     0     2
4     1     1
5     2     2
6     2     1
7     2     2
8     0     3


In [13]:
def dfToTensor(dfList, tensorList):
    for df in dfList:
        tensor = torch.FloatTensor(df.values).view(18)
        tensorList.append(tensor)

In [14]:
input_tensors = []
dfToTensor(unnormaleListe, input_tensors)

In [102]:
input_tensors[8]

tensor([3., 0., 1., 0., 4., 4., 2., 0., 0., 1., 2., 1., 1., 0., 1., 1., 4., 0.])

In [101]:
net(input_tensors[0])

tensor([0.0000, 0.3811, 0.0000], grad_fn=<ReluBackward0>)

In [106]:
def outputValues(gameList, df_stats,outputList):
    for match in gameList:
        homeTeam = match.iat[0,0]
        awayTeam = match.iat[0,1]
        output_values= df_stats[df_stats['HomeTeam']==homeTeam]
        output_values= output_values[output_values['AwayTeam']==awayTeam]
        output_values= output_values.reset_index(drop=True)
        output_values = output_values.tail(1)
        output_values = output_values.loc[:,['FTHG','FTAG']]
        homegoals = output_values.loc[:,['FTHG']].iloc[0]['FTHG']
        awaygoals = output_values.loc[:,['FTAG']].iloc[0]['FTAG']
        if homegoals > awaygoals:
            df_out=[0]
        if homegoals == awaygoals:
            df_out=[1]
        if homegoals < awaygoals:
            df_out=[2]
        outputList.append(df_out)

In [107]:
output_dfList = []
outputValues(gameList, df, output_dfList)

In [110]:
def dfToTensor_forOutput(dfList, tensorList):
    for df in dfList:
        tensor = torch.FloatTensor(df).view(1)
        tensorList.append(tensor)

In [111]:
output_tensors = []
dfToTensor_forOutput(output_dfList, output_tensors)

In [113]:
criterion = nn.CrossEntropyLoss()

In [114]:
optimizer = optim.SGD(net.parameters(),lr =0.001, momentum = 0.9)

In [120]:
for epoch in range(10):
    running_loss = 0.0
    for i in range(len(output_tensors)):
        optimizer.zero_grad()
        outputs = net(input_tensors[i])
        loss = criterion(outputs.view(1,3), output_tensors[i].long())
        loss.backward()
        optimizer.step()

In [95]:
torch.max(output_tensors[0], 0)[1]

tensor(2)

In [99]:
outputs.size()

torch.Size([3])

In [ ]:
class DataSet(dataset)